In [1]:
import pandas as pd
import numpy as np

In [2]:
snooker_raw = pd.read_html("https://en.wikipedia.org/wiki/List_of_snooker_ranking_tournaments")

In [3]:
ranking_events = snooker_raw[0]

In [4]:
# drop first six rows in ranking_events
ranking_events = ranking_events.drop(ranking_events.index[0:6])
ranking_events.head()

,R,Date,Tournament,Location,Winner,Age,Runner-up,Age.1,Sc.,Ref.
6,7,5 May 1980,World Snooker Championship,Sheffield,Cliff Thorburn (1),"32 years, 110 days",Alex Higgins (2),"31 years, 48 days",18‍–‍16,[14]
7,8,20 Apr 1981,World Snooker Championship,Sheffield,Steve Davis (1),"23 years, 241 days",Doug Mountjoy (1),"38 years, 316 days",18‍–‍12,[15]
8,9,16 May 1982,World Snooker Championship,Sheffield,Alex Higgins (1),"33 years, 59 days",Ray Reardon (1),"49 years, 220 days",18‍–‍15,[16]
9,10,10 Oct 1982,International Open,Derby,Tony Knowles (1),"27 years, 119 days",David Taylor (1),"39 years, 73 days",9‍–‍6,[17]
10,11,22 Oct 1982,Professional Players Tournament,Birmingham,Ray Reardon (5),"50 years, 14 days",Jimmy White (1),"20 years, 173 days",10‍–‍5,[18]


In [5]:
# drop columns index, age, location, and ref
ranking_events = ranking_events.drop(columns=['Age', 'Age.1', 'Location', 'Ref.'])
ranking_events.head()

,R,Date,Tournament,Winner,Runner-up,Sc.
6,7,5 May 1980,World Snooker Championship,Cliff Thorburn (1),Alex Higgins (2),18‍–‍16
7,8,20 Apr 1981,World Snooker Championship,Steve Davis (1),Doug Mountjoy (1),18‍–‍12
8,9,16 May 1982,World Snooker Championship,Alex Higgins (1),Ray Reardon (1),18‍–‍15
9,10,10 Oct 1982,International Open,Tony Knowles (1),David Taylor (1),9‍–‍6
10,11,22 Oct 1982,Professional Players Tournament,Ray Reardon (5),Jimmy White (1),10‍–‍5


In [6]:
# make new column name "season", each season is defined by tournaments from last "World Snooker Championship" (not included) to next "World Snooker Championship" (included)
ranking_events['season'] = np.nan
season = 0
for i in range(len(ranking_events)):
    ranking_events.iloc[i, 6] = season
    if ranking_events.iloc[i, 2] == 'World Snooker Championship':
        season += 1

In [21]:
# modify the winner and runner-up columns, delete the trailing brackets with numbers in it
ranking_events['Winner'] = ranking_events['Winner'].str.replace(r"\(.*\)", "", regex=True)
ranking_events['Runner-up'] = ranking_events['Runner-up'].str.replace(r"\(.*\)", "", regex=True)
ranking_events.head()

,R,Date,Tournament,Winner,Runner-up,Sc.,season
6,7,5 May 1980,World Snooker Championship,Cliff Thorburn,Alex Higgins,18‍–‍16,0.0
7,8,20 Apr 1981,World Snooker Championship,Steve Davis,Doug Mountjoy,18‍–‍12,1.0
8,9,16 May 1982,World Snooker Championship,Alex Higgins,Ray Reardon,18‍–‍15,2.0
9,10,10 Oct 1982,International Open,Tony Knowles,David Taylor,9‍–‍6,3.0
10,11,22 Oct 1982,Professional Players Tournament,Ray Reardon,Jimmy White,10‍–‍5,3.0


In [51]:
# add a new column shows winning frames, which is the number before "-" in column "Sc."
ranking_events['winning_frames'] = ranking_events['Sc.'].str.split('–').str[0]
# delete inprintable characters in the winning_frames column
def remove_non_ascii(text):
    return ''.join(i for i in text if i.isprintable())
ranking_events['winning_frames'] = ranking_events['winning_frames'].apply(remove_non_ascii)
# replace the row with "RR" as 4, which is 2021 WST Pro Series, where the winner is decided by round robin
ranking_events['winning_frames'] = ranking_events['winning_frames'].replace('RR', 4)
# make the column integer
ranking_events['winning_frames'] = ranking_events['winning_frames'].astype(int)
ranking_events.head()

,R,Date,Tournament,Winner,Runner-up,Sc.,season,winning_frames,total_winning_frames
6,7,5 May 1980,World Snooker Championship,Cliff Thorburn,Alex Higgins,18‍–‍16,0.0,18,18‍
7,8,20 Apr 1981,World Snooker Championship,Steve Davis,Doug Mountjoy,18‍–‍12,1.0,18,18‍
8,9,16 May 1982,World Snooker Championship,Alex Higgins,Ray Reardon,18‍–‍15,2.0,18,18‍
9,10,10 Oct 1982,International Open,Tony Knowles,David Taylor,9‍–‍6,3.0,9,9‍10‍18‍
10,11,22 Oct 1982,Professional Players Tournament,Ray Reardon,Jimmy White,10‍–‍5,3.0,10,9‍10‍18‍


In [53]:
# add a new column, which sum up all winning frames in each season
ranking_events['total_winning_frames'] = ranking_events.groupby('season')['winning_frames'].transform('sum')
ranking_events.tail(10)

,R,Date,Tournament,Winner,Runner-up,Sc.,season,winning_frames,total_winning_frames
417,418,21 Jan 2024,World Grand Prix,Ronnie O'Sullivan,Judd Trump,10‍–‍7,44.0,10,157
418,419,4 Feb 2024,German Masters,Judd Trump,Si Jiahui,10‍–‍5,44.0,10,157
419,420,18 Feb 2024,Welsh Open,Gary Wilson,Martin O'Donnell,9‍–‍4,44.0,9,157
420,421,25 Feb 2024,Players Championship,Mark Allen,Zhang Anda,10‍–‍8,44.0,10,157
421,422,24 Mar 2024,World Open,Judd Trump,Ding Junhui,10‍–‍4,44.0,10,157
422,423,7 Apr 2024,Tour Championship,Mark Williams,Ronnie O'Sullivan,10‍–‍5,44.0,10,157
423,424,6 May 2024,World Snooker Championship,Kyren Wilson,Jak Jones,18‍–‍14,44.0,18,157
424,425,3 Jul 2024,Championship League,Ali Carter,Jackson Page,3‍–‍1,45.0,3,23
425,426,25 Aug 2024,Xi'an Grand Prix,Kyren Wilson,Judd Trump,10‍–‍8,45.0,10,23
426,427,7 Sep 2024,Saudi Arabia Masters,Judd Trump,Mark Williams,10‍–‍9,45.0,10,23


In [54]:
# add a column, show how many tournaments in each season
ranking_events['tournaments_in_season'] = ranking_events.groupby('season')['Tournament'].transform('count')
ranking_events.tail(10)

,R,Date,Tournament,Winner,Runner-up,Sc.,season,winning_frames,total_winning_frames,tournaments_in_season
417,418,21 Jan 2024,World Grand Prix,Ronnie O'Sullivan,Judd Trump,10‍–‍7,44.0,10,157,17
418,419,4 Feb 2024,German Masters,Judd Trump,Si Jiahui,10‍–‍5,44.0,10,157,17
419,420,18 Feb 2024,Welsh Open,Gary Wilson,Martin O'Donnell,9‍–‍4,44.0,9,157,17
420,421,25 Feb 2024,Players Championship,Mark Allen,Zhang Anda,10‍–‍8,44.0,10,157,17
421,422,24 Mar 2024,World Open,Judd Trump,Ding Junhui,10‍–‍4,44.0,10,157,17
422,423,7 Apr 2024,Tour Championship,Mark Williams,Ronnie O'Sullivan,10‍–‍5,44.0,10,157,17
423,424,6 May 2024,World Snooker Championship,Kyren Wilson,Jak Jones,18‍–‍14,44.0,18,157,17
424,425,3 Jul 2024,Championship League,Ali Carter,Jackson Page,3‍–‍1,45.0,3,23,3
425,426,25 Aug 2024,Xi'an Grand Prix,Kyren Wilson,Judd Trump,10‍–‍8,45.0,10,23,3
426,427,7 Sep 2024,Saudi Arabia Masters,Judd Trump,Mark Williams,10‍–‍9,45.0,10,23,3


In [56]:
# replace the tournaments_in_season in last three rows into 18
ranking_events.iloc[-3:, 9] = 18
ranking_events.tail(5)
# replace total_winning_frames in last three rows into 167
ranking_events.iloc[-3:, 8] = 167
ranking_events.tail(5)

,R,Date,Tournament,Winner,Runner-up,Sc.,season,winning_frames,total_winning_frames,tournaments_in_season
422,423,7 Apr 2024,Tour Championship,Mark Williams,Ronnie O'Sullivan,10‍–‍5,44.0,10,157,17
423,424,6 May 2024,World Snooker Championship,Kyren Wilson,Jak Jones,18‍–‍14,44.0,18,157,17
424,425,3 Jul 2024,Championship League,Ali Carter,Jackson Page,3‍–‍1,45.0,3,167,18
425,426,25 Aug 2024,Xi'an Grand Prix,Kyren Wilson,Judd Trump,10‍–‍8,45.0,10,167,18
426,427,7 Sep 2024,Saudi Arabia Masters,Judd Trump,Mark Williams,10‍–‍9,45.0,10,167,18


In [58]:
# new column, is the winning frames divided by total winning frames
ranking_events['winning_frames_ratio'] = ranking_events['winning_frames'] / ranking_events['total_winning_frames']
ranking_events.tail(10)
# new column, is 1 divided by tournaments_in_season
ranking_events['tournaments_in_season_ratio'] = 1 / ranking_events['tournaments_in_season']

In [69]:
# group by each winner, sum up the winning_frames_ratio, rank winners by the sum
print("Total wins weighted by frames")
ranking_events.groupby('Winner')['winning_frames_ratio'].sum().sort_values(ascending=False).head(20)

Total wins weighted by frames


Winner
Steve Davis           5.504034
Ronnie O'Sullivan     4.433985
Stephen Hendry        4.348203
John Higgins          3.601208
Mark Williams         2.608666
Judd Trump            2.012335
Neil Robertson        2.012257
Mark Selby            1.859861
Ding Junhui           1.249313
Jimmy White           1.182657
Cliff Thorburn        1.148148
Peter Ebdon           1.095091
Shaun Murphy          1.014822
Alex Higgins          1.000000
John Parrott          0.978203
Mark Allen            0.761714
Ken Doherty           0.728147
Stephen Maguire       0.630607
Stuart Bingham        0.500541
Stephen Lee           0.482272
Name: winning_frames_ratio, dtype: float64

In [70]:
print("Total wins weighted by # of tournaments")
ranking_events.groupby('Winner')['tournaments_in_season_ratio'].sum().sort_values(ascending=False).head(20)

Total wins weighted by # of tournaments


Winner
Steve Davis           5.302778
Stephen Hendry        4.091667
Ronnie O'Sullivan     3.996781
John Higgins          3.463564
Mark Williams         2.594036
Neil Robertson        2.008644
Judd Trump            1.921640
Mark Selby            1.574167
Ding Junhui           1.381332
Jimmy White           1.297222
Cliff Thorburn        1.166667
Peter Ebdon           1.076190
Shaun Murphy          1.063626
Alex Higgins          1.000000
John Parrott          0.972222
Mark Allen            0.782167
Ken Doherty           0.738889
Stephen Maguire       0.685447
Stephen Lee           0.611111
Tony Knowles          0.583333
Name: tournaments_in_season_ratio, dtype: float64